In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import keras
import random 
from tqdm import tqdm


In [ ]:
import keras
import pydicom 

data_path = '../input/rsna-pneumonia-detection-challenge/'

images_path = data_path + 'stage_2_train_images/'

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,
                 list_IDs, 
                 labels, 
                 batch_size=32,
                 dim=(32,32,32), 
                 n_channels=1,
                 n_classes=10,
                 shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def read_image_from_dicom(self,ID):
        dicom_path = images_path + ID + '.dcm'
        img = pydicom.dcmread(dicom_path).pixel_array
        img = np.array(img)
        img = np.resize(img, self.dim)
        img = np.stack((img,)*3, axis=-1) 
        return img
        
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(tqdm(list_IDs_temp)):
            # Store sample
            img = self.read_image_from_dicom(ID)
            X[i,] = img
            # Store class
            y[i] = self.labels[i]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
import pickle

def read_pickle(path):    
    with open(path, 'rb') as fp:
        return pickle.load(fp)
    
data_path = '../input/samwise/'
labels_path = data_path + 'labels'
patient_ids_path = data_path + 'ids'

labels = read_pickle(labels_path)
ids = read_pickle(patient_ids_path)

In [ ]:
from keras.applications.densenet import DenseNet121
# load only conv layers, with global average pooling 
base_model = DenseNet121(include_top=False, weights='imagenet', pooling='avg')

In [ ]:
# Parameters
params = {'dim': (224,224), # default input size for VGG16
          'batch_size': 32,
          'n_classes': 3,
          'n_channels': 3,
          'shuffle': False}

feature_generator = DataGenerator(ids,labels, **params)

In [ ]:
feature_tensors = base_model.predict_generator(feature_generator, workers=4, use_multiprocessing=True)

In [ ]:
np.save('feature_vectors', feature_tensors)